# 서울시 부동산 실거래가 분석하기

# 1. 모듈 불러오기 

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import re

# 2. 데이터 불러오기

In [20]:
# pd.read_csv('data/어찌구저찌구')

## 데이터 구조
- 데이터 : 국토교통부, 통계청, 네이버 부동산, 구글맵, 부동산 114 등
- 관측치 개수 : ***개
- 변수 개수 : ***개 

## 설명 변수 (원인 : 예측값을 설명할 수 있는 변수)

### 구조적 특성
- 단지명(csv)
- 전용 면적 (csv) : 단지명이랑 전용 면적을 줄기로 파생 데이터를 붙여야됨 
- 건축년도 (csv)
- 층 수 (csv) 
- 방 수 (m. 네이버 부동산)
- 화장실 수 (m. 네이버 부동산) : 기존의 부동산 가격을 분석했던 헤도닉 모형에 공통적으로 포함되어 있음
- 난방 (m.네이버 부동산) : 2008년 원두환·김형건(난방 방식에 따른 아파트 가격 변화 분석) 참조

 
### 환경적 특성
- 크기 (총 세대 수, 동 수) (m.네이버 부동산) : 도시경제학에서 군집 효과의 크기를 의미
- 평균 주차 가능 대수 (m.네이버 부동산) : 아파트 단지 내 가구당 주차수요가 증가하면서 점차 중요한 요인으로 고려되고 있음.
- 단지 내 최고층의 높이 (m.네이버 부동산) : 단지 내의 조경
- 최저층의 높이 (m.네이버 부동산)
- 건폐율 (m.네이버 부동산) : 건축면적에 대한 대지면적의 비율
- 용적률 (m.네이버 부동산) : 총면적의 대지면적에 대한 비율 -> 재건축 시 기대할 수 있는 수익과 직접적인 연관

### 입지적 특성
- 주요 시설과의 거리 (m.네이버 부동산) 



## 반응 변수 (결과 : 예측하고자 하는 값)
- 매매가격 (국토 교통부 아파트 매매 실거래가)

# 3. 탐색적 데이터 분석 및 데이터 전처리
- X : 독립변수, 예측변수, 입력변수
- y : 종속변수, 반응변수, 출력변수

## 부동산 데이터 수집 
- 네이버 부동산 (pc) : https://new.land.naver.com/complexes/22853?ms=37.5085491,127.0104349,17&a=APT&b=A1&e=RETAIL&ad=true
- 네이버 부동산 (모바일) : https://m.land.naver.com/complex/info/22853?ptpNo=1
- complex 다음 숫자가 아파트 번호 코드
- 아파트 번호를 리스트화해서 for 반복문을 통해 크롤링

- pc 데이터는 json을 get 하는 방식으로 크롤링
- 모바일 데이터는 html 파싱하는 방식으로 크롤링

### pc 데이터로 얻을 수 있는 데이터의 정보
- complexTypeName : 건물 타입 이름
- complexType: 건물 타입
- complexName: 건물 이름
- complexNo": 건물 코드 번호
- totalHouseHoldCount : 세대수
- totalDongCount : 총 동
- useApproveYmd: 사용승인일
- minArea: 최소 면적
- maxArea: 최대 면적
- minPrice": 최소 매매가
- maxPrice": 최대 매매가
- minLeasePrice : 최소 전세가
- maxLeasePrice : 최대 전세가
- leasePerDealRate : ?
- isaleDealRestrictionCode : ?
- rebuildMembershipTransYn : ?
- latitude : 위도
- longitude : 경도

### 모바일 데이터로 얻을 수 있는 데이터의 정보 

##### 1p
- 세대수
- 최저/최고층
- 총주차대수
- 사용승인일
- 용적률
- 건폐율
- 건설사
- 난방
- 면적

##### 면적별로 다른 정보들 : 이것은 제외. 크롤링의 범위가 비효율적이고 과도하게 방대해짐.
- 공급/전용면적 (전용률)
- 방수/욕실수
- 해당면적 세대수
- 현관구조
- 관리비
- 공시가격
- 보유세

##### 5p
- 주변 편의시설/1km 이내 
- 주변 대중교통 (ex. 지하철 7 , 버스 50)

In [21]:
df17=pd.read_csv('data/아파트실거래가_17.csv',encoding='cp949')
df18=pd.read_csv('data/아파트실거래가_18.csv',encoding='cp949')
df19=pd.read_csv('data/아파트실거래가_19.csv',encoding='cp949')
df20=pd.read_csv('data/아파트실거래가_20.csv',encoding='cp949')
df21=pd.read_csv('data/아파트실거래가_21.09.csv',encoding='cp949')

In [22]:
df1 = pd.concat([df17,df18, df19, df20, df21])
df1

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201703,15,"80,000",7,1988,언주로 103,NaN
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201704,19,"85,000",2,1988,언주로 103,NaN
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,201707,8,"89,900",1,1988,언주로 103,NaN
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,201704,13,"97,500",4,1987,언주로 3,NaN
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성아파트1동~8동,79.97,201704,17,"98,000",1,1987,언주로 3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38201,서울특별시 중랑구 중화동,450,450.0,0.0,한신2,84.03,202108,21,"88,000",3,1998,동일로 752,NaN
38202,서울특별시 중랑구 중화동,274-51,274.0,51.0,한영(101),57.66,202102,27,"30,500",5,2003,동일로144길 74,NaN
38203,서울특별시 중랑구 중화동,274-75,274.0,75.0,한영(102),72.22,202105,4,"49,500",4,2003,동일로144길 74,NaN
38204,서울특별시 중랑구 중화동,274-76,274.0,76.0,한영(103),58.46,202107,1,"30,000",2,2003,동일로144길 74,NaN


In [23]:
df1.drop(['해제사유발생일'], axis=1, inplace=True)

In [24]:
df1['Addr'] = df1['시군구']+df1['번지']

In [25]:
df1.drop(['시군구', '번지', '본번', '부번', '도로명'], axis=1, inplace=True)
df1

,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,Addr
0,개포2차현대아파트(220),77.75,201703,15,"80,000",7,1988,서울특별시 강남구 개포동655-2
1,개포2차현대아파트(220),77.75,201704,19,"85,000",2,1988,서울특별시 강남구 개포동655-2
2,개포2차현대아파트(220),77.75,201707,8,"89,900",1,1988,서울특별시 강남구 개포동655-2
3,개포6차우성아파트1동~8동,79.97,201704,13,"97,500",4,1987,서울특별시 강남구 개포동658-1
4,개포6차우성아파트1동~8동,79.97,201704,17,"98,000",1,1987,서울특별시 강남구 개포동658-1
...,...,...,...,...,...,...,...,...
38201,한신2,84.03,202108,21,"88,000",3,1998,서울특별시 중랑구 중화동450
38202,한영(101),57.66,202102,27,"30,500",5,2003,서울특별시 중랑구 중화동274-51
38203,한영(102),72.22,202105,4,"49,500",4,2003,서울특별시 중랑구 중화동274-75
38204,한영(103),58.46,202107,1,"30,000",2,2003,서울특별시 중랑구 중화동274-76


In [26]:
df1.rename(columns={'단지명':'ComplexName', '전용면적(㎡)':'Area', '거래금액(만원)':'Price','층':'Floor', '건축년도' :'Construct'}, inplace=True)
df1

,ComplexName,Area,계약년월,계약일,Price,Floor,Construct,Addr
0,개포2차현대아파트(220),77.75,201703,15,"80,000",7,1988,서울특별시 강남구 개포동655-2
1,개포2차현대아파트(220),77.75,201704,19,"85,000",2,1988,서울특별시 강남구 개포동655-2
2,개포2차현대아파트(220),77.75,201707,8,"89,900",1,1988,서울특별시 강남구 개포동655-2
3,개포6차우성아파트1동~8동,79.97,201704,13,"97,500",4,1987,서울특별시 강남구 개포동658-1
4,개포6차우성아파트1동~8동,79.97,201704,17,"98,000",1,1987,서울특별시 강남구 개포동658-1
...,...,...,...,...,...,...,...,...
38201,한신2,84.03,202108,21,"88,000",3,1998,서울특별시 중랑구 중화동450
38202,한영(101),57.66,202102,27,"30,500",5,2003,서울특별시 중랑구 중화동274-51
38203,한영(102),72.22,202105,4,"49,500",4,2003,서울특별시 중랑구 중화동274-75
38204,한영(103),58.46,202107,1,"30,000",2,2003,서울특별시 중랑구 중화동274-76


In [27]:
df=pd.read_csv('data/df_result.csv',encoding='cp949')
df

,gu,dong,apt,title,household,floor_low,floor_top,parking,floor_area,building_cover,...,subway,child,preschool,school,parking_lot,market,convenience,laundry,bank,hospital
0,1168000000,1168010300,8928,LG개포자이,212,20,22,502,250.0,18.0,...,380m,183m,414m,112m,266m,325m,244m,38m,302m,1001
1,1168000000,1168010300,119219,개포래미안포레스트,"2,296",7,35,"3,961",249.0,21.0,...,1001,131m,131m,269m,437m,495m,133m,324m,281m,1001
2,1168000000,1168010300,140057,개포비버리하임(도시형),29,6,6,15,199.0,57.0,...,1001,161m,169m,223m,120m,359m,17m,90m,131m,1001
3,1168000000,1168010300,107513,개포상지리츠빌,18,10,10,41,202.0,31.0,...,80m,609m,534m,192m,525m,509m,409m,83m,459m,1001
4,1168000000,1168010300,103385,개포현대(200동),72,9,9,NaN,157.0,17.0,...,769m,15m,269m,109m,435m,170m,115m,152m,107m,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8368,1126000000,1126010300,104106,태솔,19,3,6,20,199.0,47.0,...,275m,127m,131m,1001,46m,112m,75m,69m,110m,1001
8369,1126000000,1126010300,102174,한솔e,35,7,7,35,0.0,0.0,...,387m,122m,165m,126m,184m,167m,78m,89m,388m,1001
8370,1126000000,1126010300,824,한신,"1,544",23,27,"1,548",376.0,20.0,...,240m,59m,102m,123m,229m,216m,179m,113m,230m,1001
8371,1126000000,1126010300,18519,한영,72,5,7,73,294.0,58.0,...,512m,163m,306m,145m,326m,88m,180m,166m,380m,1001


In [28]:
df.rename(columns={'title':'ComplexName'}, inplace=True)
df

,gu,dong,apt,ComplexName,household,floor_low,floor_top,parking,floor_area,building_cover,...,subway,child,preschool,school,parking_lot,market,convenience,laundry,bank,hospital
0,1168000000,1168010300,8928,LG개포자이,212,20,22,502,250.0,18.0,...,380m,183m,414m,112m,266m,325m,244m,38m,302m,1001
1,1168000000,1168010300,119219,개포래미안포레스트,"2,296",7,35,"3,961",249.0,21.0,...,1001,131m,131m,269m,437m,495m,133m,324m,281m,1001
2,1168000000,1168010300,140057,개포비버리하임(도시형),29,6,6,15,199.0,57.0,...,1001,161m,169m,223m,120m,359m,17m,90m,131m,1001
3,1168000000,1168010300,107513,개포상지리츠빌,18,10,10,41,202.0,31.0,...,80m,609m,534m,192m,525m,509m,409m,83m,459m,1001
4,1168000000,1168010300,103385,개포현대(200동),72,9,9,NaN,157.0,17.0,...,769m,15m,269m,109m,435m,170m,115m,152m,107m,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8368,1126000000,1126010300,104106,태솔,19,3,6,20,199.0,47.0,...,275m,127m,131m,1001,46m,112m,75m,69m,110m,1001
8369,1126000000,1126010300,102174,한솔e,35,7,7,35,0.0,0.0,...,387m,122m,165m,126m,184m,167m,78m,89m,388m,1001
8370,1126000000,1126010300,824,한신,"1,544",23,27,"1,548",376.0,20.0,...,240m,59m,102m,123m,229m,216m,179m,113m,230m,1001
8371,1126000000,1126010300,18519,한영,72,5,7,73,294.0,58.0,...,512m,163m,306m,145m,326m,88m,180m,166m,380m,1001


In [29]:
df1 = pd.merge(df1, df, how='inner', on='ComplexName')
df1

,ComplexName,Area,계약년월,계약일,Price,Floor,Construct,Addr,gu,dong,...,subway,child,preschool,school,parking_lot,market,convenience,laundry,bank,hospital
0,개포현대6차,134.86,201706,26,"79,700",6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
1,개포현대6차,132.63,201707,8,"80,000",5,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
2,개포현대6차,132.63,201901,13,"102,000",6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
3,경남2차,182.20,201702,11,"183,000",11,1984,서울특별시 강남구 개포동649,1141000000,1141011000,...,452m,177m,1001,230m,114m,1001,132m,135m,208m,1001
4,경남2차,91.91,201702,11,"120,500",14,1984,서울특별시 강남구 개포동649,1141000000,1141011000,...,452m,177m,1001,230m,114m,1001,132m,135m,208m,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628136,이지마루종로,26.33,202109,27,"34,800",11,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346m,181m,252m,99m,111m,1001,40m,209m,241m,1001
628137,이지마루종로,24.44,202109,28,"34,800",8,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346m,181m,252m,99m,111m,1001,40m,209m,241m,1001
628138,이지마루종로,26.65,202109,29,"35,000",5,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346m,181m,252m,99m,111m,1001,40m,209m,241m,1001
628139,청구드림,84.99,202103,27,"70,000",1,2002,서울특별시 중구 신당동290-90,1114000000,1114016200,...,183m,24m,406m,87m,86m,215m,23m,151m,172m,1001


In [30]:
X = df1.drop('Price', axis=1)
y = df1['Price']

display(X.head(3))

,ComplexName,Area,계약년월,계약일,Floor,Construct,Addr,gu,dong,apt,...,subway,child,preschool,school,parking_lot,market,convenience,laundry,bank,hospital
0,개포현대6차,134.86,201706,26,6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,10863,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
1,개포현대6차,132.63,201707,8,5,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,10863,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
2,개포현대6차,132.63,201901,13,6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,10863,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001


## 여기부터 전처리


- 평수 , 'm'빼기 , 계약날짜 컬럼 완료
- 정책 더미 아직

In [31]:
df1

,ComplexName,Area,계약년월,계약일,Price,Floor,Construct,Addr,gu,dong,...,subway,child,preschool,school,parking_lot,market,convenience,laundry,bank,hospital
0,개포현대6차,134.86,201706,26,"79,700",6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
1,개포현대6차,132.63,201707,8,"80,000",5,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
2,개포현대6차,132.63,201901,13,"102,000",6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
3,경남2차,182.20,201702,11,"183,000",11,1984,서울특별시 강남구 개포동649,1141000000,1141011000,...,452m,177m,1001,230m,114m,1001,132m,135m,208m,1001
4,경남2차,91.91,201702,11,"120,500",14,1984,서울특별시 강남구 개포동649,1141000000,1141011000,...,452m,177m,1001,230m,114m,1001,132m,135m,208m,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628136,이지마루종로,26.33,202109,27,"34,800",11,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346m,181m,252m,99m,111m,1001,40m,209m,241m,1001
628137,이지마루종로,24.44,202109,28,"34,800",8,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346m,181m,252m,99m,111m,1001,40m,209m,241m,1001
628138,이지마루종로,26.65,202109,29,"35,000",5,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346m,181m,252m,99m,111m,1001,40m,209m,241m,1001
628139,청구드림,84.99,202103,27,"70,000",1,2002,서울특별시 중구 신당동290-90,1114000000,1114016200,...,183m,24m,406m,87m,86m,215m,23m,151m,172m,1001


### 제곱미터 -> 평수 

In [32]:
df1['Area']=round( df1['Area']/3.3)

In [33]:
df1

,ComplexName,Area,계약년월,계약일,Price,Floor,Construct,Addr,gu,dong,...,subway,child,preschool,school,parking_lot,market,convenience,laundry,bank,hospital
0,개포현대6차,41.0,201706,26,"79,700",6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
1,개포현대6차,40.0,201707,8,"80,000",5,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
2,개포현대6차,40.0,201901,13,"102,000",6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
3,경남2차,55.0,201702,11,"183,000",11,1984,서울특별시 강남구 개포동649,1141000000,1141011000,...,452m,177m,1001,230m,114m,1001,132m,135m,208m,1001
4,경남2차,28.0,201702,11,"120,500",14,1984,서울특별시 강남구 개포동649,1141000000,1141011000,...,452m,177m,1001,230m,114m,1001,132m,135m,208m,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628136,이지마루종로,8.0,202109,27,"34,800",11,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346m,181m,252m,99m,111m,1001,40m,209m,241m,1001
628137,이지마루종로,7.0,202109,28,"34,800",8,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346m,181m,252m,99m,111m,1001,40m,209m,241m,1001
628138,이지마루종로,8.0,202109,29,"35,000",5,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346m,181m,252m,99m,111m,1001,40m,209m,241m,1001
628139,청구드림,26.0,202103,27,"70,000",1,2002,서울특별시 중구 신당동290-90,1114000000,1114016200,...,183m,24m,406m,87m,86m,215m,23m,151m,172m,1001


### m 빼기

In [34]:
# m 빼기
columns = ['bus','subway','child','preschool','school','parking_lot','market','convenience','laundry','bank','hospital']
for col in columns:
    df1[col] = df1[col].str.strip("m")

In [35]:
df1

,ComplexName,Area,계약년월,계약일,Price,Floor,Construct,Addr,gu,dong,...,subway,child,preschool,school,parking_lot,market,convenience,laundry,bank,hospital
0,개포현대6차,41.0,201706,26,"79,700",6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812,66,479,489,155,313,88,202,172,1001
1,개포현대6차,40.0,201707,8,"80,000",5,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812,66,479,489,155,313,88,202,172,1001
2,개포현대6차,40.0,201901,13,"102,000",6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,...,812,66,479,489,155,313,88,202,172,1001
3,경남2차,55.0,201702,11,"183,000",11,1984,서울특별시 강남구 개포동649,1141000000,1141011000,...,452,177,1001,230,114,1001,132,135,208,1001
4,경남2차,28.0,201702,11,"120,500",14,1984,서울특별시 강남구 개포동649,1141000000,1141011000,...,452,177,1001,230,114,1001,132,135,208,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628136,이지마루종로,8.0,202109,27,"34,800",11,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346,181,252,99,111,1001,40,209,241,1001
628137,이지마루종로,7.0,202109,28,"34,800",8,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346,181,252,99,111,1001,40,209,241,1001
628138,이지마루종로,8.0,202109,29,"35,000",5,2021,서울특별시 종로구 효제동126-2,1111000000,1111016200,...,346,181,252,99,111,1001,40,209,241,1001
628139,청구드림,26.0,202103,27,"70,000",1,2002,서울특별시 중구 신당동290-90,1114000000,1114016200,...,183,24,406,87,86,215,23,151,172,1001


### 날짜
- 계약년월+ 계약일 --> 계약날짜 컬럼으로 생성 

In [36]:
df1['계약날짜']=df1['계약년월'].astype(str)  +df1['계약일'].astype(str)
df1['계약날짜']=pd.to_datetime(df1["계약날짜"], format='%Y%m%d')

In [37]:
df1[["계약년월","계약일","계약날짜"]]

,계약년월,계약일,계약날짜
0,201706,26,2017-06-26
1,201707,8,2017-07-08
2,201901,13,2019-01-13
3,201702,11,2017-02-11
4,201702,11,2017-02-11
...,...,...,...
628136,202109,27,2021-09-27
628137,202109,28,2021-09-28
628138,202109,29,2021-09-29
628139,202103,27,2021-03-27


In [38]:
df1.shape

(628141, 31)

### 정책 적용

In [39]:
import datetime

In [47]:
policy_1_date = datetime.date(2017, 6, 19)
policy_2_date = datetime.date(2017, 8, 2)
policy_3_date = datetime.date(2017, 9, 5)
policy_4_date = datetime.date(2017, 10, 24)
policy_5_date = datetime.date(2018, 9, 13)
policy_6_date = datetime.date(2018, 12, 19)
# 정책 6번 날짜정보 애매함
# https://namu.wiki/w/3%EA%B8%B0%20%EC%8B%A0%EB%8F%84%EC%8B%9C?rev=197
policy_7_date = datetime.date(2019, 12, 16)
policy_8_date = datetime.date(2020, 6, 17)
policy_9_date = datetime.date(2020, 7, 10)
policy_10_date = datetime.date(2020, 8, 4)

policy_date = [policy_1_date, policy_2_date,policy_3_date,policy_4_date,policy_5_date,
             policy_6_date,policy_7_date,policy_8_date,policy_9_date,policy_10_date]

In [48]:
policy_date

[datetime.date(2017, 6, 19),
 datetime.date(2017, 8, 2),
 datetime.date(2017, 9, 5),
 datetime.date(2017, 10, 24),
 datetime.date(2018, 9, 13),
 datetime.date(2018, 12, 19),
 datetime.date(2019, 12, 16),
 datetime.date(2020, 6, 17),
 datetime.date(2020, 7, 10),
 datetime.date(2020, 8, 4)]

In [49]:
policy_col = ["policy_1", "policy_2","policy_3","policy_4","policy_5",
              "policy_6","policy_7","policy_8","policy_9","policy_10"]

In [50]:
for date, col in zip(policy_date, policy_col):
    df1[col] = df1["계약날짜"].apply(lambda x: 1 if x >= date else 0)

In [51]:
df1[policy_col].head()

,policy_1,policy_2,policy_3,policy_4,policy_5,policy_6,policy_7,policy_8,policy_9,policy_10
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0


In [52]:
df1["계약날짜"].head()

0   2017-06-26
1   2017-07-08
2   2019-01-13
3   2017-02-11
4   2017-02-11
Name: 계약날짜, dtype: datetime64[ns]

In [53]:
df1.shape

(628141, 41)

### 결측치, 데이터 확인

In [97]:
X.head()

,ComplexName,Area,계약년월,계약일,Floor,Construct,Addr,gu,dong,apt,...,subway,child,preschool,school,parking_lot,market,convenience,laundry,bank,hospital
0,개포현대6차,134.86,201706,26,6,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,10863,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
1,개포현대6차,132.63,201707,8,5,1993,서울특별시 강남구 개포동1242-2,1168000000,1168010300,10863,...,812m,66m,479m,489m,155m,313m,88m,202m,172m,1001
2,경남2차,182.20,201702,11,11,1984,서울특별시 강남구 개포동649,1141000000,1141011000,3485,...,452m,177m,1001,230m,114m,1001,132m,135m,208m,1001
3,경남2차,91.91,201702,11,14,1984,서울특별시 강남구 개포동649,1141000000,1141011000,3485,...,452m,177m,1001,230m,114m,1001,132m,135m,208m,1001
4,경남2차,186.16,201703,11,9,1984,서울특별시 강남구 개포동649,1141000000,1141011000,3485,...,452m,177m,1001,230m,114m,1001,132m,135m,208m,1001


In [98]:
y.tail()

173339    34,900
173340    34,200
173341    34,500
173342    37,000
173343    33,000
Name: Price, dtype: object

In [99]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173344 entries, 0 to 173343
Data columns (total 29 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ComplexName     173344 non-null  object 
 1   Area            173344 non-null  float64
 2   계약년월            173344 non-null  int64  
 3   계약일             173344 non-null  int64  
 4   Floor           173344 non-null  int64  
 5   Construct       173344 non-null  int64  
 6   Addr            173266 non-null  object 
 7   gu              173344 non-null  int64  
 8   dong            173344 non-null  int64  
 9   apt             173344 non-null  int64  
 10  household       173344 non-null  object 
 11  floor_low       173344 non-null  int64  
 12  floor_top       173344 non-null  int64  
 13  parking         154558 non-null  object 
 14  floor_area      173344 non-null  float64
 15  building_cover  173344 non-null  float64
 16  construction    169279 non-null  object 
 17  heating   

In [100]:
X.describe()

,Area,계약년월,계약일,Floor,Construct,gu,dong,apt,floor_low,floor_top,floor_area,building_cover
count,173344.000000,173344.000000,173344.000000,173344.000000,173344.000000,1.733440e+05,1.733440e+05,173344.000000,173344.000000,173344.000000,173344.000000,173344.000000
mean,80.817205,201706.604128,16.310239,9.226590,1996.092262,1.143887e+09,1.143898e+09,15791.717337,10.811017,15.526796,248.080418,22.735047
std,25.268324,3.065423,8.769398,5.783202,7.804197,1.819308e+07,1.819261e+07,32232.261047,4.996643,6.520340,114.978758,14.609435
min,12.850000,201701.000000,1.000000,-1.000000,1966.000000,1.111000e+09,1.111010e+09,1.000000,1.000000,3.000000,0.000000,0.000000
25%,59.950000,201704.000000,9.000000,5.000000,1992.000000,1.126000e+09,1.126011e+09,736.000000,7.000000,12.000000,207.000000,17.000000
50%,84.570000,201706.000000,17.000000,9.000000,1997.000000,1.144000e+09,1.144011e+09,3070.000000,10.000000,15.000000,251.000000,22.000000
75%,84.971000,201709.000000,24.000000,13.000000,2001.000000,1.159000e+09,1.159011e+09,11213.000000,14.000000,20.000000,302.000000,28.000000
max,257.850000,201712.000000,31.000000,57.000000,2017.000000,1.174000e+09,1.174011e+09,143891.000000,44.000000,58.000000,1477.000000,91.000000


### Train / Test 데이터 분리
- Training 데이터 70% / Testing 데이터 30% 나누기

In [107]:
# Train Set / Test Set 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify=y, test_size=0.3, random_state=2021) 
                                                    # stratify = y 옵션의 의미 : 극단적으로 1, 0이 나뉘는 경우 방지

# 데이터 컬럼 단위 정규화 하기
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

print(f'Train Set dimension is {X_train.shape}')
print(f'Test Set dimension is {X_test.shape}')

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

# 4. 모델링 
- 정규화 (Normalization)
- 랜덤 포레스트
- 부스팅
- 등등..

## 4.1 정규화 모델
- Lasso, Ridge, ElasticNet

### 4.1.1 Linear Regression vs Lasso vs Ridge 계수 비교

### 4.1.2 K-fold Cross Validation

### 4.1.3 K-fold Cross Validation (K-fold 교차 검증)으로 하이퍼 파라미터 찾기

### 4.1.4 모델 비교 선택

### 4.1.5 최종 모델 학습 

#### 예측 결과 plotting

#### 계수값을 변수 중요도로 생각하기

## 4.2 랜덤 포레스트 모델

### 4.2.1 단순 모델링 (사용자가 하이퍼 파라미터 정의)

### 4.2.2 단일 하이퍼 파라미터에 대한 Grid Search

### 4.2.3 전체 하이퍼 파라미터에 대한 Grid Search (파라미터 최적화)
- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

## 4.3 부스팅 모델
- Adaboost : 이전 분류기가 예측하지 못한 데이터에 높은 가중치를 둠으로써 다음 차례의 모델은 예측할 수 있도록 설계된 모델
- GBM : 이전 분류기가 예측하지 못한 에러(gradient)를 다음 차례의 모델이 학습하도록 설계된 모델
- LightGBM : 분류기 모델이 의사결정 규칙을 찾을 때, 모든 feature와 data index에 대해 탐색하는 비효율성 개선
- Catboost : 범주형 변수가 많을 때 효율적이며 target leakage와 prediction shift 문제점 해결

### 4.3.1 하이퍼파라미터 소개
- Learing rate : 부스팅에 대한 Regularization 효과를 지님
- N_estimators : 부스팅에 활용될 예측 모델의 총 개수
- Subsample : 다음 시점의 약 분류기가 학습할 데이터의 비율을 의미하며 0과 1사이의 값을 가짐
- Max depth : 약한 분류기 모델의 복잡도를 결정하는 값으로 주로 작은 값(1~5)에서 결정됨

### 4.3.2 학습 

### 4.3.3 학습 결과 확인

### 4.3.4 하이퍼파라미터 세밀하게 탐색 (your option)

### 4.3.5 테스팅 데이터에 대한 최적 예측 성능 관찰

# 5. 알고리즘끼리 비교 / 앙상블

# EOD